# Segmenting and Clustering the neighborhoods in Toronto

## <b> Question 1

In [1]:
#importing all the required libraries
import numpy as np
import pandas as pd
import requests
import sys

### Scraping the wikipedia page for data on postal codes


In [2]:
import pip
if int(pip.__version__.split('.')[0])>9:
        from pip._internal import main
else:
        from pip import main
def install(package):
       main(['install', package])

install('BeautifulSoup4')

In [3]:
#import beautifulsoup and csv
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

In [4]:
#reading data from wikipedia page into dataframe
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML code.
soup = BeautifulSoup(raw_wikipedia_page,'html.parser')

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

### Creating the Dataframe

In [6]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

In [7]:
#data is transformed into pandas dataframe
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

In [8]:
#only consider the assigned boroughs
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'html.parser')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

In [9]:
#drop the cells that have unassigned boroughs
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [10]:
#new dataframe df1 after reseting the index
df1 = df.reset_index()

In [11]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          103 non-null    int64 
 1   PostalCode     103 non-null    object
 2   Borough        103 non-null    object
 3   Neighbourhood  103 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


(103, 4)

In [12]:
#grouping the dataframe on the basis of postal code
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.info()
df2.shape

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Borough        103 non-null    object
 1   Neighbourhood  103 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


(103, 2)

In [13]:
#assigning no assigned neighbourhood cells bporoughs as the neighbourhoods
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

In [14]:
df3 = df2.reset_index()

In [15]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [16]:
# pandas dataframe on the information regarding the neighbourhoods of toronto
df3.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [17]:
print('dataframe created')

dataframe created


In [18]:
df3.info()
df3.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


(103, 3)

## Question 2

### Creating the toronto neighbourhood dataframe with latitude,longitude values.

In [19]:
conda install -c conda-forge geopy

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [20]:
#import geocoders and Nominatim
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='toronto_explorer')
#test for a location 
location = geolocator.geocode("Toronto, North York, Parkwoods")
print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 124974741, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [21]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})
df_geo = df3

In [22]:
#add a column callled adress that is the combination of postalcode,borough,neighbourhood
geolocator = Nominatim(user_agent='toronto_explorer')

df_geo['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geo.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West..."
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [23]:
df_geo = df3

In [24]:
#drop the boroughs that are not assigned
df_geo.drop(df_geo[df_geo['Borough']=="Notassigned"].index,axis=0, inplace=True)
df_geo.info()
df_geo.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
 3   address        103 non-null    object
dtypes: object(4)
memory usage: 4.0+ KB


(103, 4)

In [25]:
#saving the df_geo dataframe values to "geo" csv file
df_geo.to_csv('geo.csv')

In [26]:
#opening geopy.csv file for reading
df3.to_csv('geopy.csv')
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)

### Creating a CSV file

In [27]:
#creating a csv file for saving postalcode,neighbourhood,borough,latitude,longitude data
import numpy as np
import csv


PostalCode = None
Borough = None
Neighbourhood = None
latData = None
longData = None

LAT_Woburn = 43.7598243
LONG_Woburn = -79.2252908
LAT_Malvern = 43.8091955
LONG_Malvern = -79.2217008
LAT_Highland_Creek = 43.7901172
LONG_Highland_Creek = -79.1733344
LAT_Morningside = 43.7826012
LONG_Morningside = -79.2049579

 
PostalCode = np.array(['M1H','M1B','M1C','M1G '])
Borough = np.array(['Scarborough','Scarborough','Scarborough','Scarborough'])
Neighbourhood = np.array(['Woburn','Malvern','Highland_Creek','Morningside'])
latData = np.array([43.7598243,43.8091955, 43.7901172 , 43.7826012])
longData = np.array([-79.2252908,-79.2217008,-79.1733344, -79.2049579 ])

with open('data.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow('ABXYZ')
    for a,b,x,y,z in np.nditer([ PostalCode.T, Borough.T, Neighbourhood.T, latData.T, longData.T], order='C'):
        writer.writerow([a,b,x,y,z])    

In [28]:
#Reading the contents of the csv file
import csv

with open('data.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row['A'], row['B'],row['X'], row['Y'], row['Z'])

M1H Scarborough Woburn 43.7598243 -79.2252908
M1B Scarborough Malvern 43.8091955 -79.2217008
M1C Scarborough Highland_Creek 43.7901172 -79.1733344
M1G  Scarborough Morningside 43.7826012 -79.2049579


In [29]:
pd.read_csv('data.csv') 

,A,B,X,Y,Z
0,M1H,Scarborough,Woburn,43.759824,-79.225291
1,M1B,Scarborough,Malvern,43.809196,-79.221701
2,M1C,Scarborough,Highland_Creek,43.790117,-79.173334
3,M1G,Scarborough,Morningside,43.782601,-79.204958


### Creating Dataframes

In [30]:
df_geopy=df3
type(df_geopy)

pandas.core.frame.DataFrame

In [31]:
#adding address column to the geopy dataframe
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Malvern, Rouge","M1B,Scarborough,Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C,Scarborough,Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E,Scarborough,Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae"


In [32]:
#adding the coordinates column to the dataframe
df_geopy['Coordinates'] =df_geopy['address'].apply(Nominatim(user_agent='toronto_explorer').geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M1B,Scarborough,"Malvern, Rouge","M1B,Scarborough,Malvern, Rouge",None
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C,Scarborough,Rouge Hill, Port Union, Highla...",None
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E,Scarborough,Guildwood, Morningside, West Hill",None
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None


In [33]:
#Adding latitude,Longitude values to the dataframe
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Malvern, Rouge","M1B,Scarborough,Malvern, Rouge",None,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C,Scarborough,Rouge Hill, Port Union, Highla...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E,Scarborough,Guildwood, Morningside, West Hill",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None,NaN,NaN


In [34]:
df_geopy.to_csv('geo_loc_py.csv')

In [35]:
#Saving the data n geopy csv file into data1 datafrrame
data1 = pd.read_csv("geopy.csv") 
data1.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High..."
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West..."
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [36]:
#Saving the geographical coordinates csv file in data2 dataframe
data2 = pd.read_csv("http://cocl.us/Geospatial_data")
data2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
data2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Genrating the final dataframe with lat/long values

In [37]:
#Merging the gtwi dataframes into one data3 dataframe
data3 = pd.merge(data2, data1, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data3.head()

,PostalCode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge"
1,M1C,43.784535,-79.160497,1,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High..."
2,M1E,43.763573,-79.188711,2,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West..."
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [38]:
#rearranging the columns
cols = data1.columns.tolist()
cols

['Unnamed: 0', 'PostalCode', 'Borough', 'Neighbourhood', 'address']

In [39]:
new_column_order = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [40]:
#generate the required dataframe by rearranging the coulmn order
data3 = data3[new_column_order]
data3.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [41]:
#Sorting the values so as to match the values in the eample dataframe
sorted_df = data3.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])
sorted_df.reset_index(inplace=True)
sorted_cols =sorted_df.columns.tolist()
example_dataframe = sorted_df[new_column_order]
example_dataframe.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,NorthYork,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
5,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
7,M6K,WestToronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
8,M7Y,EastToronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
9,M5V,DowntownToronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [42]:
print('Required Dataframe Generated.')

Required Dataframe Generated.


## Question 3

### Clustering and Exploring neighbourhood data 

In [43]:
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as color
from sklearn.cluster import KMeans
import folium

In [44]:
#saving csv files data into example_dataframe
example_dataframe.to_csv('sorted_geoloc.csv')
example_dataframe.to_json(path_or_buf='geo_toronto.json', orient='table')

In [45]:
#loading data into dataframe called toronto_data
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [46]:
#duplicating the dataframe
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]

{'index': 0,
 'PostalCode': 'M1S',
 'Borough': 'Scarborough',
 'Neighbourhood': 'Agincourt',
 'Latitude': 43.7942003,
 'Longitude': -79.2620294}

In [47]:
example_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,NorthYork,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,"Bedford Park, Lawrence Manor East",43.733283,-79.419750


In [48]:
#exploring the metadata about the dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format( len(example_dataframe['Borough'].unique()), example_dataframe.shape[0] ))

The dataframe has 10 boroughs and 103 neighborhoods.


In [49]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



### Generating and Exploring Maps in toronto 

In [50]:
import folium

toronto_n = example_dataframe.sample(n=20)
map_toronto = folium.Map(location=[toronto_n['Latitude'].mean(), 
                                toronto_n['Longitude'].mean()], 
                      zoom_start=10)
for row in toronto_n.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Neighbourhood))

    
map_toronto 
map_toronto.save("map_toronto20.html")

In [51]:
#creating amap on toronto neighbourhoods
address = 'Toronto, CA'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [52]:
#generating the map and exploring it
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(example_dataframe['Latitude'], example_dataframe['Longitude'], example_dataframe['Borough'], example_dataframe['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods


### Exploring map and data in a specific neighbourhood (york)

In [53]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of York, Toronto are 43.6896191, -79.479188.


In [54]:
york_data = example_dataframe[example_dataframe['Borough'] == 'York'].reset_index(drop=True)
york_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [55]:
map_york_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york_toronto)  
    
map_york_toronto


In [56]:
#entering personal foursquare credentials
CLIENT_ID = 'ABWAGUVX2FZ5JAAGIU5YNKQ2N4D1B0RNXF5Y1M4SBMGB0LR0' # your Foursquare ID

CLIENT_SECRET = 'FYYPFTE4X1VEPAY3JQIWUFJ2WH4KXTZWK4IBDTANQH5FMY02' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

In [57]:
#setting values for the locations in york
neighbourhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Caledonia-Fairbanks are 43.6890256, -79.453512.


In [58]:
#generating a URL
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(

CLIENT_ID, 

CLIENT_SECRET, 

VERSION, 

neighbourhood_latitude, 

neighbourhood_longitude, 

radius, 

LIMIT)


In [59]:
york_results = requests.get(url).json()

### Getting common venues in york

In [60]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [61]:
york_venues = york_results['response']['groups'][0]['items']
    
york_nearby_venues = json_normalize(york_venues) # flatten JSON

york_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
york_nearby_venues = york_nearby_venues.loc[:, york_filtered_columns]


york_nearby_venues['venue.categories'] = york_nearby_venues.apply(get_category_type, axis=1)


york_nearby_venues.columns = [col.split(".")[-1] for col in york_nearby_venues.columns]

york_nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Nairn Park,Park,43.690654,-79.456300
1,Maximum Woman,Women's Store,43.690651,-79.456333
2,Fairbanks Pool,Pool,43.691959,-79.448922
3,Fairbank Memorial Park,Park,43.692028,-79.448924


In [62]:
#generating similar url for every venue in york
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
   
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
     
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
#creating the dataframe of the most common venues in york
york_venues = getNearbyVenues(names=york_data['Neighbourhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Caledonia-Fairbanks
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humewood-Cedarvale
Runnymede, The Junction North
Weston


In [64]:
york_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park
1,Caledonia-Fairbanks,43.689026,-79.453512,Maximum Woman,43.690651,-79.456333,Women's Store
2,Caledonia-Fairbanks,43.689026,-79.453512,Fairbanks Pool,43.691959,-79.448922,Pool
3,Caledonia-Fairbanks,43.689026,-79.453512,Fairbank Memorial Park,43.692028,-79.448924,Park
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,Subway,43.690218,-79.474050,Sandwich Place


### Clustering data into categories

In [65]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
york_fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[york_fixed_columns]

york_onehot.head()

,Neighbourhood,Breakfast Spot,Bus Line,Convenience Store,Discount Store,Fast Food Restaurant,Field,Fried Chicken Joint,Hockey Arena,Park,Pizza Place,Pool,Sandwich Place,Tennis Court,Trail,Women's Store
0,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [66]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped.head()

,Neighbourhood,Breakfast Spot,Bus Line,Convenience Store,Discount Store,Fast Food Restaurant,Field,Fried Chicken Joint,Hockey Arena,Park,Pizza Place,Pool,Sandwich Place,Tennis Court,Trail,Women's Store
0,Caledonia-Fairbanks,0.000000,0.000000,0.0,0.00,0.00,0.00,0.00,0.00,0.5,0.000000,0.25,0.00,0.00,0.00,0.25
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0.000000,0.0,0.25,0.25,0.00,0.25,0.00,0.0,0.000000,0.00,0.25,0.00,0.00,0.00
2,Humewood-Cedarvale,0.000000,0.000000,0.0,0.00,0.00,0.25,0.00,0.25,0.0,0.000000,0.00,0.00,0.25,0.25,0.00
3,"Runnymede, The Junction North",0.333333,0.333333,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.333333,0.00,0.00,0.00,0.00,0.00
4,Weston,0.000000,0.000000,0.5,0.00,0.00,0.00,0.00,0.00,0.5,0.000000,0.00,0.00,0.00,0.00,0.00


In [67]:
#fiding the top common venues
num_top_venues = 3

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    york_temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    york_temp.columns = ['venue','freq']
    york_temp = york_temp.iloc[1:]
    york_temp['freq'] = york_temp['freq'].astype(float)
    york_temp = york_temp.round({'freq': 2})
    print(york_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
           venue  freq
0           Park  0.50
1           Pool  0.25
2  Women's Store  0.25


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                  venue  freq
0        Discount Store  0.25
1  Fast Food Restaurant  0.25
2   Fried Chicken Joint  0.25


----Humewood-Cedarvale----
          venue  freq
0         Field  0.25
1  Hockey Arena  0.25
2  Tennis Court  0.25


----Runnymede, The Junction North----
            venue  freq
0  Breakfast Spot  0.33
1        Bus Line  0.33
2     Pizza Place  0.33


----Weston----
               venue  freq
0  Convenience Store   0.5
1               Park   0.5
2     Breakfast Spot   0.0




In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
#creating a dataframe of the most common venues in york
num_top_venues = 17

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
york_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)

york_neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue


In [70]:
york_neighbourhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

york_neighbourhoods_venues_sorted.head(2)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Caledonia-Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
for ind in np.arange(york_grouped.shape[0]):
    york_neighbourhoods_venues_sorted.iloc[ind, 3:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

york_neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Caledonia-Fairbanks,NaN,NaN,Park,Women's Store,Pool,Trail,Tennis Court,Sandwich Place,Pizza Place,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Convenience Store,Bus Line,Breakfast Spot
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",NaN,NaN,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Women's Store,Trail,Tennis Court,Pool,Pizza Place,Park,Hockey Arena,Field,Convenience Store,Bus Line,Breakfast Spot
2,Humewood-Cedarvale,NaN,NaN,Trail,Tennis Court,Hockey Arena,Field,Women's Store,Sandwich Place,Pool,Pizza Place,Park,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Convenience Store,Bus Line,Breakfast Spot
3,"Runnymede, The Junction North",NaN,NaN,Pizza Place,Bus Line,Breakfast Spot,Women's Store,Trail,Tennis Court,Sandwich Place,Pool,Park,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Convenience Store
4,Weston,NaN,NaN,Park,Convenience Store,Women's Store,Trail,Tennis Court,Sandwich Place,Pool,Pizza Place,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Bus Line,Breakfast Spot


### Generating clusters

In [72]:
#running kmeans clustering
k = 2

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

york_kmeans = KMeans(n_clusters=k, random_state=0).fit(york_grouped_clustering)

york_kmeans.labels_[0:5] 

array([0, 1, 1, 1, 0], dtype=int32)

In [73]:
york_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', york_kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(york_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,NaN,NaN,Park,Women's Store,...,Sandwich Place,Pizza Place,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Convenience Store,Bus Line,Breakfast Spot
1,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,1,NaN,NaN,Sandwich Place,Fried Chicken Joint,...,Trail,Tennis Court,Pool,Pizza Place,Park,Hockey Arena,Field,Convenience Store,Bus Line,Breakfast Spot
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1,NaN,NaN,Trail,Tennis Court,...,Sandwich Place,Pool,Pizza Place,Park,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Convenience Store,Bus Line,Breakfast Spot
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262,1,NaN,NaN,Pizza Place,Bus Line,...,Tennis Court,Sandwich Place,Pool,Park,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Convenience Store
4,M9N,York,Weston,43.706876,-79.518188,0,NaN,NaN,Park,Convenience Store,...,Sandwich Place,Pool,Pizza Place,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Bus Line,Breakfast Spot


In [76]:
#differentiating the clusters
york_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [color.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(york_map_clusters)
       
york_map_clusters

york_map_clusters.save("york_map_clusters.html")


### Cluster Number 1 (red) :

In [77]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,York,0,NaN,NaN,Park,Women's Store,Pool,Trail,Tennis Court,Sandwich Place,Pizza Place,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Convenience Store,Bus Line,Breakfast Spot
4,York,0,NaN,NaN,Park,Convenience Store,Women's Store,Trail,Tennis Court,Sandwich Place,Pool,Pizza Place,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Bus Line,Breakfast Spot


### Cluster Number 2 (blue) :

In [80]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
1,York,1,NaN,NaN,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Women's Store,Trail,Tennis Court,Pool,Pizza Place,Park,Hockey Arena,Field,Convenience Store,Bus Line,Breakfast Spot
2,York,1,NaN,NaN,Trail,Tennis Court,Hockey Arena,Field,Women's Store,Sandwich Place,Pool,Pizza Place,Park,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Convenience Store,Bus Line,Breakfast Spot
3,York,1,NaN,NaN,Pizza Place,Bus Line,Breakfast Spot,Women's Store,Trail,Tennis Court,Sandwich Place,Pool,Park,Hockey Arena,Fried Chicken Joint,Field,Fast Food Restaurant,Discount Store,Convenience Store


In [81]:
print('Clustering done')

Clustering done
